# Imports

In [1]:
import os

from random import shuffle

import pandas as pd

import cv2

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import tensorflow as tf
import tflearn

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

Instructions for updating:
non-resource variables are not supported in the long term


# Global Independent Variables (Hyper-Parameters) - Identify The Model

In [2]:
# Labels Vectors
categories = {
'Basketball':  np.array([1, 0, 0, 0, 0, 0]),
'Football':    np.array([0, 1, 0, 0, 0, 0]),
'Rowing':      np.array([0, 0, 1, 0, 0, 0]),
'Swimming':    np.array([0, 0, 0, 1, 0, 0]),
'Tennis':      np.array([0, 0, 0, 0, 1, 0]),
'Yoga':        np.array([0, 0, 0, 0, 0, 1])
}


In [3]:
TRAIN_DIR = './Input/Train/'
TEST_DIR = './Input/NTest/'
# TEST_DIR = './Input/Test/'
IMG_SIZE = 50
LR = 0.001
MODEL_NAME = 'sports-image-classification-cnn'

# Helpers

In [4]:
def create_label(image_name):
    """ Create an one-hot encoded vector from image name """
    word_label = image_name.split('_')[0]
    return categories[word_label]

In [5]:
def create_train_data():
    """To bring Train Images from the State of Raw Files into Structured Numpy Array with its Label attatched with it.
    Returns:
        np.ndarray: Training Images [np.array(img), its Label]
    """
    training_data = []
    
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
    
        img_data = cv2.imread(path, 1)                                   # 0: Read Img as Grayscale
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE)) 
        
        training_data.append([np.array(img_data), create_label(img)])
        
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    
    return training_data

In [6]:
def create_test_data():
    testing_data=[]
    
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        
        img_data = cv2.imread(path, 1)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        
        testing_data.append([img, np.array(img_data)])
        
    np.save('test_data.npy', np.array(testing_data))
    
    return np.array(testing_data)

In [7]:
def prob_to_output(probabilities):
    predictions = []
    
    for probability in probabilities:
        max_index = np.where(probability == max(probability))[0][0]
        predict = np.zeros(shape=probability.shape)
        predict[max_index] = 1.0
        predictions.append(predict)
        
    return predictions

In [8]:
def output_to_label(output_vectors):
    return [np.where(vector == 1)[0][0] for vector in output_vectors]

# Data Preparation

In [9]:
# # Real-time image preprocessing
# img_prep = ImagePreprocessing()
# # 
# img_prep.add_featurewise_zero_center()
# img_prep.add_featurewise_stdnorm()

In [10]:
# # Real-time data augmentation
# img_aug = ImageAugmentation()
# #
# img_aug.add_random_flip_leftright()
# img_aug.add_random_rotation(max_angle=25.)

In [11]:
# Do not do it Twice!
if (os.path.exists('train_data.npy')):
    train_data =np.load('train_data.npy',allow_pickle=True)
else: 
    train_data = create_train_data()

In [12]:
# Seprate Labels from Raw Image Matrices
X_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_train = [i[1] for i in train_data]

In [13]:
print(Y_train[0].shape)
print(X_train.shape)

(6,)
(1681, 50, 50, 3)


# Model سِره فـ أضعف خلقه

## Construction

In [14]:
tf.compat.v1.reset_default_graph()

In [15]:
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')    
# 
conv1 = conv_2d(conv_input, 32, 5, activation='relu')    # 32 Feature Maps\Filters ~ Kernel Size = 5x5
pool1 = max_pool_2d(conv1, 5)

conv2 = conv_2d(pool1, 64, 5, activation='relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool2, 128, 5, activation='relu')
pool3 = max_pool_2d(conv3, 5)

conv4 = conv_2d(pool3, 64, 5, activation='relu')
pool4 = max_pool_2d(conv4, 5)

conv5 = conv_2d(pool4, 32, 5, activation='relu')
pool5 = max_pool_2d(conv5, 5)
#  
fully_layer = fully_connected(pool5, 1024, activation='relu')    # 1024 Neurons
fully_layer = dropout(fully_layer, 0.5)    # ~ dropout one-half
# 
cnn_layers = fully_connected(fully_layer, 6, activation='softmax')    # 6 Categories
# 
cnn_layers = regression(cnn_layers, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
# 
secret_in_weakest = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Runtime Session

In [16]:
secret_in_weakest.fit(X_train, Y_train, n_epoch=10, shuffle=True, validation_set=0.2,
          show_metric=True, batch_size=96, run_id=MODEL_NAME)

Training Step: 699  | total loss: 0.32442 | time: 7.497s
| Adam | epoch: 050 | loss: 0.32442 - acc: 0.9702 -- iter: 1248/1344
Training Step: 700  | total loss: 0.29351 | time: 9.078s
| Adam | epoch: 050 | loss: 0.29351 - acc: 0.9731 | val_loss: 0.87212 - val_acc: 0.7953 -- iter: 1344/1344
--


## Testing & Competition Submissions

In [17]:
# Do not do it Twice!
if (os.path.exists('test_data.npy')):
    test_data =np.load('test_data.npy',allow_pickle=True)
else: 
    test_data = create_test_data()

100%|██████████| 93/93 [00:00<00:00, 214.08it/s]
/home/ahmed/anaconda3/envs/Neural/lib/python3.6/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':
/home/ahmed/anaconda3/envs/Neural/lib/python3.6/site-packages/ipykernel_launcher.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [18]:
X_test = np.array([i[1] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
image_names = [i[0] for i in test_data]

In [19]:
X_test.shape

(93, 50, 50, 3)

In [20]:
secret_in_weakest

In [21]:
predictions = secret_in_weakest.predict(X_test)

In [22]:
results = prob_to_output(predictions)

In [23]:
labels = output_to_label(results)
labels[:10]

[5, 4, 3, 4, 4, 1, 5, 4, 4, 3]

In [24]:
submission=pd.DataFrame()
submission['image_name'] = image_names
submission['label'] = labels
submission.to_csv("submission_prctical.csv",index=False)

In [25]:
pd.read_csv('./submission_prctical.csv').head(2)

,image_name,label
0,75.jpg,5
1,37.jpg,4


# Tensorboard and Visualization

# End of Model

# Packages Installations

In [26]:
# ! pip install tflearn

In [27]:
# REQUIREMENTS MAY BE NEEDED
"""
conda create -n NeuralHandsOn python=3.6
pip install tensorflow==2.4
pip install opencv-python
pip install matplotlib
pip install scipy
conda install tflearn
conda install -c conda-forge tqdm
conda install -c anaconda qt
"""

'\nconda create -n NeuralHandsOn python=3.6\npip install tensorflow==2.4\npip install opencv-python\npip install matplotlib\npip install scipy\nconda install tflearn\nconda install -c conda-forge tqdm\nconda install -c anaconda qt\n'

In [28]:
# prevent scientific notation
np.set_printoptions(suppress=True)

# Garbage Codes

In [29]:
# # Model Training ~ Do not train same Thing Twice
# if (os.path.exists('secret_in_weakest.tfl.meta')):
#     secret_in_weakest.load('./secret_in_weakest.tfl')
# else:
#     secret_in_weakest.fit(X_train, Y_train, n_epoch=50, shuffle=True, validation_set=0.2,
#           show_metric=True, batch_size=96, run_id=MODEL_NAME)
#     secret_in_weakest.save('secret_in_weakest.tfl')

In [30]:
# conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input',
#                         data_preprocessing=img_prep,
#                         data_augmentation=img_aug